In [1]:
"""
# 🎯 Notebook 05 : Recommandation d'Offres d'Emploi

**Objectif** : Matcher un CV avec toutes les offres et recommander les plus pertinentes

**Pipeline** :
1. Charger le CV et ses compétences extraites (Étape 2)
2. Charger toutes les offres d'emploi (Étape 4)
3. Utiliser le JobMatcher pour calculer les scores
4. Trier et recommander les TOP offres

**Modules utilisés** :
- `src/cv_parser.py` : Parser de CV
- `src/job_matcher.py` : Système de matching sémantique
"""

"\n# 🎯 Notebook 05 : Recommandation d'Offres d'Emploi\n\n**Objectif** : Matcher un CV avec toutes les offres et recommander les plus pertinentes\n\n**Pipeline** :\n1. Charger le CV et ses compétences extraites (Étape 2)\n2. Charger toutes les offres d'emploi (Étape 4)\n3. Utiliser le JobMatcher pour calculer les scores\n4. Trier et recommander les TOP offres\n\n**Modules utilisés** :\n- `src/cv_parser.py` : Parser de CV\n- `src/job_matcher.py` : Système de matching sémantique\n"

In [2]:
# Imports
import sys
import json
from pathlib import Path
import numpy as np
from typing import List, Dict

# Ajouter le dossier racine
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

print(f"📁 Projet : {project_root}")
print(f"📂 Notebooks : {Path().absolute()}")

# Importer nos modules custom
try:
    from src.job_matcher import JobMatcher, load_matcher
    print("✅ JobMatcher importé depuis src/job_matcher.py")
except ImportError as e:
    print(f"❌ Erreur d'import : {e}")
    print("⚠️  Vérifiez que src/job_matcher.py existe")

print("\n✅ Imports terminés")

📁 Projet : c:\Users\rober\OneDrive\Bureau\PFE
📂 Notebooks : c:\Users\rober\OneDrive\Bureau\PFE\notebooks
✅ JobMatcher importé depuis src/job_matcher.py

✅ Imports terminés


In [3]:
# Charger les compétences extraites du CV (Étape 2)
skills_path = project_root / "outputs" / "extracted_skills_simple.json"

if not skills_path.exists():
    print("❌ Fichier compétences non trouvé")
    print(f"⚠️  Chemin attendu : {skills_path}")
    print("⚠️  Exécutez d'abord 02_skills_extraction_simple.ipynb")
    cv_skills = []
else:
    with open(skills_path, 'r', encoding='utf-8') as f:
        cv_data = json.load(f)
    
    cv_skills = cv_data['technical_skills']
    cv_soft_skills = cv_data.get('soft_skills', [])
    
    print(f"✅ Compétences CV chargées")
    print(f"   • Fichier : {cv_data.get('cv_file', 'N/A')}")
    print(f"   • Date extraction : {cv_data.get('extraction_date', 'N/A')}")
    print(f"   • Compétences techniques : {len(cv_skills)}")
    print(f"   • Soft skills : {len(cv_soft_skills)}")
    print(f"\n📋 Exemples de compétences :")
    for skill in cv_skills[:10]:
        print(f"   • {skill}")

✅ Compétences CV chargées
   • Fichier : RESUME ROBERT UNG.pdf
   • Date extraction : 2026-01-25
   • Compétences techniques : 24
   • Soft skills : 5

📋 Exemples de compétences :
   • .net
   • artificial intelligence
   • big data
   • c
   • c#
   • c++
   • data analysis
   • data science
   • deep learning
   • docker


In [4]:
# Charger toutes les offres d'emploi (Étape 4)
jobs_dir = project_root / "data" / "jobs"
all_jobs = []

if not jobs_dir.exists():
    print(f"❌ Dossier jobs/ non trouvé : {jobs_dir}")
    print("⚠️  Exécutez d'abord 04_job_generation.ipynb")
else:
    # Charger le dataset complet
    dataset_path = jobs_dir / "jobs_dataset.json"
    
    if dataset_path.exists():
        with open(dataset_path, 'r', encoding='utf-8') as f:
            dataset = json.load(f)
        
        all_jobs = dataset['jobs']
        metadata = dataset['metadata']
        
        print(f"✅ Offres d'emploi chargées")
        print(f"   • Fichier : {dataset_path.name}")
        print(f"   • Total offres : {metadata['total_jobs']}")
        print(f"   • Généré le : {metadata['generated_date']}")
        print(f"   • Catégories : {', '.join(metadata['categories'])}")
        
        # Statistiques par catégorie
        print(f"\n📊 Répartition par catégorie :")
        for category in metadata['categories']:
            count = len([j for j in all_jobs if j['category'] == category])
            print(f"   • {category:20s} : {count} offres")
    else:
        print(f"❌ Fichier {dataset_path.name} non trouvé")
        print("⚠️  Exécutez 04_job_generation.ipynb")

✅ Offres d'emploi chargées
   • Fichier : jobs_dataset.json
   • Total offres : 25
   • Généré le : 2026-01-25 18:21:43
   • Catégories : ml_engineer, data_scientist, python_developer, devops_engineer, frontend_developer

📊 Répartition par catégorie :
   • ml_engineer          : 5 offres
   • data_scientist       : 5 offres
   • python_developer     : 5 offres
   • devops_engineer      : 5 offres
   • frontend_developer   : 5 offres


In [14]:
# Charger le modèle de matching sémantique
print("\n🔍 Initialisation du JobMatcher...")
print("⏳ Chargement du modèle all-mpnet-base-v2 (première fois : ~420 MB)")

try:
    matcher = load_matcher()
    print(f"✅ JobMatcher initialisé")
    print(f"   • Modèle : all-mpnet-base-v2")
    print(f"   • Dimension embeddings : 768")
except Exception as e:
    print(f"❌ Erreur : {e}")
    matcher = None

2026-02-07 14:06:50,499 - src.job_matcher - INFO - Initialisation du JobMatcher...
2026-02-07 14:06:50,506 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2026-02-07 14:06:50,507 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2



🔍 Initialisation du JobMatcher...
⏳ Chargement du modèle all-mpnet-base-v2 (première fois : ~420 MB)


2026-02-07 14:06:52,312 - src.job_matcher - INFO - ✅ skills_reference.json chargé (171 skills)
2026-02-07 14:06:52,315 - src.job_matcher - INFO - ✅ JobMatcher initialisé avec all-mpnet-base-v2


✅ JobMatcher initialisé
   • Modèle : all-mpnet-base-v2
   • Dimension embeddings : 768


In [17]:
# Calculer le score pour chaque offre avec le JobMatcher
if not cv_skills:
    print("❌ Aucune compétence CV chargée")
elif not all_jobs:
    print("❌ Aucune offre chargée")
elif matcher is None:
    print("❌ JobMatcher non initialisé")
else:
    print("\n🔍 Calcul des scores de matching...")
    print(f"⏳ Analyse de {len(all_jobs)} offres avec {len(cv_skills)} compétences...")
    print("   (Temps estimé : 30-60 secondes)\n")
    
    # Utiliser la méthode rank_jobs du JobMatcher
    recommendations = matcher.rank_jobs(cv_skills, all_jobs)
    
    print(f"✅ Analyse terminée !")
    print(f"   • {len(recommendations)} offres scorées")
    print(f"   • Meilleur score : {recommendations[0]['score']:.1f}%")  # ✅ 'score'
    print(f"   • Score moyen : {sum(r['score'] for r in recommendations) / len(recommendations):.1f}%")
    print(f"   • Pire score : {recommendations[-1]['score']:.1f}%")

2026-02-07 14:11:27,569 - src.job_matcher - INFO - 🎯 Ranking 25 offres avec 24 compétences
2026-02-07 14:11:27,570 - src.job_matcher - INFO - 📊 Scoring basé uniquement sur les compétences
2026-02-07 14:11:27,588 - src.job_matcher - INFO - 💼 Skills extraits de l'offre : 17
2026-02-07 14:11:27,590 - src.job_matcher - INFO -    Exemples : numpy, pandas, scikitlearn
2026-02-07 14:11:27,591 - src.job_matcher - INFO - 🔍 Matching 17 skills offre ↔ 24 skills CV



🔍 Calcul des scores de matching...
⏳ Analyse de 25 offres avec 24 compétences...
   (Temps estimé : 30-60 secondes)



2026-02-07 14:11:29,119 - src.job_matcher - INFO - ✅ Coverage: 76.5% | Quality: 80.1% | Score: 78.3%
2026-02-07 14:11:29,128 - src.job_matcher - INFO - 💼 Skills extraits de l'offre : 16
2026-02-07 14:11:29,129 - src.job_matcher - INFO -    Exemples : cnns, architectures modernes, deeplearning
2026-02-07 14:11:29,129 - src.job_matcher - INFO - 🔍 Matching 16 skills offre ↔ 24 skills CV
2026-02-07 14:11:30,025 - src.job_matcher - INFO - ✅ Coverage: 68.8% | Quality: 67.9% | Score: 68.3%
2026-02-07 14:11:30,031 - src.job_matcher - INFO - 💼 Skills extraits de l'offre : 14
2026-02-07 14:11:30,032 - src.job_matcher - INFO -    Exemples : tokenization, embeddings, transformers
2026-02-07 14:11:30,032 - src.job_matcher - INFO - 🔍 Matching 14 skills offre ↔ 24 skills CV
2026-02-07 14:11:30,920 - src.job_matcher - INFO - ✅ Coverage: 42.9% | Quality: 76.1% | Score: 59.5%
2026-02-07 14:11:30,928 - src.job_matcher - INFO - 💼 Skills extraits de l'offre : 16
2026-02-07 14:11:30,930 - src.job_matcher - 

✅ Analyse terminée !
   • 25 offres scorées
   • Meilleur score : 94.3%
   • Score moyen : 65.6%
   • Pire score : 0.0%


In [18]:
# Afficher les 10 meilleures recommandations
if 'recommendations' in locals() and recommendations:
    print("\n" + "="*70)
    print("🏆 TOP 10 OFFRES RECOMMANDÉES")
    print("="*70)
    
    for i, job in enumerate(recommendations[:10], 1):
        # Emoji selon le score (✅ utiliser 'score')
        if job['score'] >= 70:  # ✅
            emoji = "🟢"
            label = "EXCELLENT MATCH"
        elif job['score'] >= 50:  # ✅
            emoji = "🟡"
            label = "BON MATCH"
        elif job['score'] >= 40:  # ✅
            emoji = "🟠"
            label = "MATCH MOYEN"
        else:
            emoji = "🔴"
            label = "MATCH FAIBLE"
        
        print(f"\n{emoji} #{i} - {job['title']} ({label})")
        print("-" * 70)
        print(f"   🏢 Entreprise      : {job['company']}")
        print(f"   📍 Localisation    : {job['location']} {'🏠 (Remote)' if job['remote_ok'] else ''}")
        print(f"   💼 Type            : {job['type']} | Expérience : {job['experience']}")
        print(f"   💰 Salaire         : {job['salary']}")
        print(f"   👥 Candidats       : {job['applicants']}")
        
        # ✅ Afficher UNIQUEMENT les métriques disponibles
        print(f"\n   📊 SCORES DÉTAILLÉS :")
        print(f"      • Score Global : {job['score']:.1f}% ⭐")  # ✅
        print(f"      • Coverage     : {job['skills_details']['coverage']:.1f}% (compétences couvertes)")  # ✅
        print(f"      • Quality      : {job['skills_details']['quality']:.1f}% (qualité des matchs)")  # ✅
        print(f"      • Matchs       : {job['skills_details']['covered_count']}/{job['skills_details']['total_required']} skills")  # ✅
        
        # ✅ Afficher top_matches (liste de dicts, pas liste simple)
        print(f"\n   🎯 TOP COMPÉTENCES MATCHÉES :")
        if job['skills_details']['top_matches']:  # ✅ 'top_matches' au lieu de 'top_skills'
            for match in job['skills_details']['top_matches']:
                print(f"      ✓ {match['job_skill']:20s} ↔ {match['cv_skill']:20s} ({match['similarity']:.1f}%)")
        else:
            print("      ⚠️  Aucun match fort trouvé")
        
        print(f"\n   🔧 Compétences requises (extrait) :")
        for req in job['requirements'][:3]:
            print(f"      • {req}")
        
        print(f"\n   🔗 Postuler : {job['url']}")


🏆 TOP 10 OFFRES RECOMMANDÉES

🟢 #1 - Data Scientist - HealthTech (EXCELLENT MATCH)
----------------------------------------------------------------------
   🏢 Entreprise      : MedAI
   📍 Localisation    : Toulouse, France 
   💼 Type            : CDI | Expérience : 2-4 ans
   💰 Salaire         : 48-58K€
   👥 Candidats       : 134

   📊 SCORES DÉTAILLÉS :
      • Score Global : 94.3% ⭐
      • Coverage     : 100.0% (compétences couvertes)
      • Quality      : 88.6% (qualité des matchs)
      • Matchs       : 5/5 skills

   🎯 TOP COMPÉTENCES MATCHÉES :
      ✓ python               ↔ python               (100.0%)
      ✓ artificialintelligence ↔ artificialintelligence (100.0%)
      ✓ machinelearning      ↔ machinelearning      (100.0%)
      ✓ deeplearning         ↔ deeplearning         (100.0%)
      ✓ computervision       ↔ machinelearning      (43.0%)

   🔧 Compétences requises (extrait) :
      • Machine Learning et/ou Deep Learning
      • Python avancé
      • Statistiques

   🔗

In [19]:
# Statistiques sur les recommandations
if 'recommendations' in locals() and recommendations:
    print("\n" + "="*70)
    print("📊 STATISTIQUES DES RECOMMANDATIONS")
    print("="*70)
    
    # Distribution des scores (✅ utiliser 'score')
    excellent_match = [j for j in recommendations if j['score'] >= 70]  # ✅
    good_match = [j for j in recommendations if 50 <= j['score'] < 70]  # ✅
    medium_match = [j for j in recommendations if 40 <= j['score'] < 50]  # ✅
    low_match = [j for j in recommendations if j['score'] < 40]  # ✅
    
    print(f"\n🎯 Distribution des matches :")
    print(f"   • Excellent match (≥70%)  : {len(excellent_match):2d} offres 🟢")
    print(f"   • Bon match (50-70%)      : {len(good_match):2d} offres 🟡")
    print(f"   • Match moyen (40-50%)    : {len(medium_match):2d} offres 🟠")
    print(f"   • Match faible (<40%)     : {len(low_match):2d} offres 🔴")
    
    # Score moyen par catégorie
    print(f"\n📂 Score moyen par catégorie :")
    categories = set(job.get('category', 'unknown') for job in all_jobs)
    
    category_scores = {}
    for category in categories:
        cat_jobs = [r for r in recommendations 
                    if any(j['job_id'] == r['job_id'] and j.get('category') == category 
                          for j in all_jobs)]
        
        if cat_jobs:
            avg_score = sum(j['score'] for j in cat_jobs) / len(cat_jobs)  # ✅
            category_scores[category] = avg_score
    
    # Trier par score décroissant
    for category, avg_score in sorted(category_scores.items(), key=lambda x: x[1], reverse=True):
        bar_length = int(avg_score / 5)
        bar = "█" * bar_length
        print(f"   • {category:25s} : {avg_score:5.1f}% {bar}")
    
    # Remote vs On-site
    remote_jobs = [j for j in recommendations if j['remote_ok']]
    onsite_jobs = [j for j in recommendations if not j['remote_ok']]
    
    print(f"\n🏠 Remote vs On-site :")
    if remote_jobs:
        avg_remote = sum(j['score'] for j in remote_jobs) / len(remote_jobs)  # ✅
        print(f"   • Offres remote  ({len(remote_jobs):2d}) : {avg_remote:.1f}% 🏠")
    
    if onsite_jobs:
        avg_onsite = sum(j['score'] for j in onsite_jobs) / len(onsite_jobs)  # ✅
        print(f"   • Offres on-site ({len(onsite_jobs):2d}) : {avg_onsite:.1f}% 🏢")
    
    # Offres par niveau d'expérience
    print(f"\n💼 Recommandations par niveau :")
    exp_levels = {}
    for job in recommendations:
        exp = job['experience']
        if exp not in exp_levels:
            exp_levels[exp] = {'count': 0, 'avg_score': 0}
        exp_levels[exp]['count'] += 1
        exp_levels[exp]['avg_score'] += job['score']  # ✅
    
    for exp, data in sorted(exp_levels.items(), key=lambda x: x[1]['avg_score'], reverse=True):
        avg = data['avg_score'] / data['count']
        print(f"   • {exp:15s} : {data['count']:2d} offres | Score moyen : {avg:.1f}%")


📊 STATISTIQUES DES RECOMMANDATIONS

🎯 Distribution des matches :
   • Excellent match (≥70%)  : 11 offres 🟢
   • Bon match (50-70%)      : 11 offres 🟡
   • Match moyen (40-50%)    :  1 offres 🟠
   • Match faible (<40%)     :  2 offres 🔴

📂 Score moyen par catégorie :
   • data_scientist            :  88.2% █████████████████
   • ml_engineer               :  71.9% ██████████████
   • python_developer          :  70.2% ██████████████
   • devops_engineer           :  50.5% ██████████
   • frontend_developer        :  47.4% █████████

🏠 Remote vs On-site :
   • Offres remote  ( 8) : 66.7% 🏠
   • Offres on-site (17) : 65.1% 🏢

💼 Recommandations par niveau :
   • 2-4 ans         :  8 offres | Score moyen : 69.4%
   • 0-2 ans         :  5 offres | Score moyen : 74.6%
   • 1-3 ans         :  5 offres | Score moyen : 65.9%
   • 3-5 ans         :  4 offres | Score moyen : 77.0%
   • 4-6 ans         :  2 offres | Score moyen : 37.5%
   • 5-7 ans         :  1 offres | Score moyen : 0.0%


In [20]:
# Analyser les compétences les plus demandées
if 'recommendations' in locals() and recommendations:
    print("\n" + "="*70)
    print("🔧 COMPÉTENCES LES PLUS DEMANDÉES (TOP offres)")
    print("="*70)
    
    # Collecter toutes les compétences des 10 meilleures offres
    from collections import Counter
    
    all_requirements = []
    for job in recommendations[:10]:
        all_requirements.extend(job['requirements'])
    
    # Compter les occurrences
    skill_counts = Counter(all_requirements)
    
    print(f"\n📋 TOP 15 compétences recherchées :")
    for skill, count in skill_counts.most_common(15):
        bar = "█" * count
        print(f"   • {skill:40s} : {bar} ({count})")
    
    # Compétences manquantes (dans les requirements mais pas dans le CV)
    print(f"\n⚠️  Compétences à développer (présentes dans les TOP offres) :")
    
    cv_skills_lower = [s.lower() for s in cv_skills]
    missing_skills = set()
    
    for job in recommendations[:10]:
        for req in job['requirements']:
            req_lower = req.lower()
            # Vérification simple (pas de matching sémantique ici)
            if not any(cv_skill in req_lower or req_lower in cv_skill 
                      for cv_skill in cv_skills_lower):
                missing_skills.add(req)
    
    if missing_skills:
        for skill in sorted(missing_skills)[:10]:
            print(f"   • {skill}")
    else:
        print("   ✅ Vous avez toutes les compétences principales !")


🔧 COMPÉTENCES LES PLUS DEMANDÉES (TOP offres)

📋 TOP 15 compétences recherchées :
   • SQL                                      : ███ (3)
   • Statistiques                             : ██ (2)
   • Kubernetes en production                 : ██ (2)
   • Leadership technique                     : ██ (2)
   • Machine Learning et/ou Deep Learning     : █ (1)
   • Python avancé                            : █ (1)
   • Éthique et RGPD                          : █ (1)
   • Communication avec médecins              : █ (1)
   • Python (pandas, numpy, matplotlib)       : █ (1)
   • Machine Learning (scikit-learn)          : █ (1)
   • Communication (présentation de résultats) : █ (1)
   • Machine Learning (classification, regression) : █ (1)
   • Time series analysis                     : █ (1)
   • Python (pandas, scikit-learn)            : █ (1)
   • Statistiques avancées                    : █ (1)

⚠️  Compétences à développer (présentes dans les TOP offres) :
   • APIs RESTful
   • MLOps
   